#### Step 1: Do the following variable declarations:

In [1]:
# Create a variable to return to the index of the closest point from a given set of points
bestIndex=0
# Create a variable to find the number of clusters to find
K=5
# Create a variable to find the threshold distance between iterations at which we decide we are done
convergeDist = .1
# Create a varialbe to assign temporary distance to positive infinity
tempDist = float("+inf")

#### Step 2: Create a function to add two coordinates

In [2]:
def sumOfPoints(point1,point2):
    return point1[0]+point2[0],point1[1]+point2[1]

#### Step 3: Create a function to find the squared distance between two points

In [3]:
def squaredDistanceBetweenPoints(point1,point2):
    return (point1[0]-point2[0]) **2 + (point1[1]-point2[1]) ** 2

#### Step 4: Create a function to find the index of the point from a given set set of points which is closest to a given point

In [4]:
def closestPoint(point,points):
    bestIndex=0
    closestDistanceBetweenPoints = float("+inf")
    for i in xrange(0,len(points)):
        distanceBetweenPoints=squaredDistanceBetweenPoints(point,points[i])
        if distanceBetweenPoints < closestDistanceBetweenPoints:
            closestDistanceBetweenPoints=distanceBetweenPoints
            bestIndex=i
    return bestIndex

#### Step 5:  RDD to map all latitudes and longitudes

In [9]:
points=sc.textFile("file:/home/cloudera/training_materials/data/userinfo.csv")\
       .map(lambda line:line.split(","))\
       .map(lambda fields: (float(fields[1]),float(fields[2])))\
       .persist()

#### Step 6: Choose random points taken to decide the number of clusters

In [10]:
kPoints = points.takeSample(False, K, 34)
print "Starting points:", kPoints

Starting points: [(14.6042, 120.9822), (11.2741, 106.35879), (6.15154, -75.61658), (29.6175, 121.405), (-33.8678, 151.2073)]


#### Step 7: Loop over the elements to find the cluster centres 

In [11]:
# Loop till temporary distance is greater than convergent distance
while tempDist > convergeDist:
    # Index of the closest point from a given point
    closest = points.map(lambda p : (closestPoint(p, kPoints), (p, 1)))
    # Add the coordinates and the number of occurences by Key
    pointStats = closest.reduceByKey(lambda (point1,n1),(point2,n2):  (sumOfPoints(point1,point2),n1+n2) )
    # New point by calculating the average of each closest point
    newPoints = pointStats.map(lambda (i,(point,n)): (i,[point[0]/n,point[1]/n])).collect()
    # Total of the distanceStep  between the current points and new points
    tempDist=0
    for  (i,point) in newPoints: tempDist += squaredDistanceBetweenPoints(kPoints[i],point)
    print "Distance between iterations:",tempDist
    # New points to the kPoints array for the next iteration
    for (i, point) in newPoints: kPoints[i] = point

Distance between iterations: 4164.14839164
Distance between iterations: 1009.68366739
Distance between iterations: 695.247151757
Distance between iterations: 15.4464697554
Distance between iterations: 0.0


#### Step 8: Print all the center points

In [12]:
print "Final center points: " + str(kPoints)


Final center points: [[-0.7504521264367817, 114.1482109195403], [39.57328149870801, 18.896898501292004], [3.7804638613861363, -73.56115465346534], [32.5512856223176, 110.29270836909878], [-35.442235, 163.52865]]
